## Import necessary python modules

In [ ]:
from copy import deepcopy as dc

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from streakimageIR.streakimage import StreakImage
import import_ipynb
from helpful_functions import *

import glob
import os
import re

# Correct an individual dataset

## Import Data

### Load background

In [ ]:
path_to_bg =  "enter path to bg here" # path relativ to notebook folder
bg = StreakImage(path_to_bg)

### Load image

In [ ]:
path_to_img = "enter path to image here" # path relativ to notebook folder
image = StreakImage(path_to_img, bg=bg)

### Correction

Try different parameters for the corner widths and length to find a set of parameters that works well with all datasets.
You may need to load different images.

### Background subtraction

In [ ]:
# defining variables vor better readability
corner1_width = 100
corner2_width = -50
corner_length = 50

def cornerbg(image: StreakImage, ci1=10, ci2=-20, t_max=10):
    corner_1=image.data.iloc[:t_max,:ci1]
    corner_2=image.data.iloc[:t_max,ci2:]
    o1 = corner_1.mean().mean()
    o2 = corner_2.mean().mean()
    offset = (o1+o2)/2
    image.data -= offset
    
def cornerbg_and_plot(image: StreakImage, ci1=100, ci2=-50, t_max=50):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12.8,4.8))
    
    plot_heatmap(image.data, fig, ax1)
    
    origin1 = ax1.get_xlim()[0], ax1.get_ylim()[1]
    box1 = plt.Rectangle(origin1, ci1, t_max, fc="none", ec="r", lw=2)
    ax1.add_patch(box1)
    origin2 = ax1.get_xlim()[1], ax1.get_ylim()[1]
    box2 = plt.Rectangle(origin2, ci2, t_max, fc="none", ec="r", lw=2)
    ax1.add_patch(box2)
    
    cornerbg(image=image, ci1=corner1_width, ci2=-corner2_width, t_max=corner_length)
    plot_heatmap(image.data, fig, ax2)
    
cornerbg_and_plot(image)


In [ ]:
def apply_other_correction(image):
    image.apply_gain_correction()
    image.apply_exp_correction()
#     image.apply_camera_correction() # missing camera correction
#     image.apply_mono_correction() # missing mono correction
    
def correct_and_plot(image: StreakImage):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12.8,4.8))
    
    plot_heatmap(image.data, fig, ax1)
    apply_other_correction(image=image)
    plot_heatmap(image.data, fig, ax2)
    
correct_and_plot(image)

In [ ]:
def get_spec_and_trans(image):
    spec = np.sum(image.data.loc[:,:], axis=0)
    trans = np.sum(image.data.loc[:,:], axis=1)
    
    spec.columns=["wavelength", "intensity"]
    return spec, trans

def get_and_plot_spec_and_trans(image):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12.8,4.8))
    spec, trans = get_spec_and_trans(image)
    
    ax1.plot(spec)
    ax2.plot(trans)
    
    ax1.set_xlim(left=780, right=900)
    ax2.set_yscale("log")

get_and_plot_spec_and_trans(image)

In [ ]:
def export_spec_and_trans(image, file_path):
    spec, trans = get_spec_and_trans(image)
    
    with open(f"{file_path}_spec.txt", 'w') as file:
        spec.to_csv(file, header=["Intensity (arb. units)"], index_label="Wavelength (nm)")
    with open(f"{file_path}_trans.txt", 'w') as file:
        spec.to_csv(file, header=["Intensity (arb. units)"], index_label="Time (ps)")

## Apply corrections to all images in a folder

In [ ]:
folder_path = "enter path to folder here"

file_list = glob.glob(os.path.join(folder_path, "*.img"))
bg_list = glob.glob(os.path.join(folder_path, "BG*"))

image_list = [path for path in file_list if path not in bg_list]

The fourth and fith last lines perform the corrections and plot images as above.
Run the cell as is first and check your corrections.
Then comment out the two lines and uncomment the three last lines to perform all correction and export all files.

In [ ]:
for image in image_list:
    fname = os.path.basename(image)
    rex = r"_(BGS[0-9][0-9][A-Z])_"
    bg_name = re.search(rex, fname).groups()[0]
    
    bg_path = [path for path in bg_list if bg_name in path][0]
    
    bg = StreakImage(bg_path)
    image = StreakImage(image_list[0], bg=bg)
    
    cornerbg_and_plot(image)
    correct_and_plot(image)
    get_and_plot_spec_and_trans(image)

#     cornerbg(image)
#     apply_other_correction(image)
#     export_spec_and_trans(image, f"{folder_path}/export/{fname}")